<a href="https://colab.research.google.com/github/suyashchaturvedi26/PE-Valuation-Model/blob/main/PE_Valuation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install html-table-parser-python3

In [ ]:
pip install yfinance --upgrade --no-cache-dir

In [ ]:
#importing all the necessary libraries
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [ ]:
nifty_500=pd.read_excel('ind_nifty500list.xlsx')
nifty_500['URL']='https://www.screener.in/company/'+ nifty_500['Symbol'] +'/consolidated/#profit-loss'
nifty_500['Symbol']=nifty_500['Symbol']+'.NS'
nifty_500

## **EPS FETCH**

In [ ]:
def url_get_contents(url):
    
  
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    return f.read()

In [ ]:
def EPS(pos1,pos2):
  
  
  EPS1=pd.DataFrame()
  EPS2=pd.DataFrame()

  for index,row in nifty_500.iterrows():
    try:
      xhtml = url_get_contents(row['URL']).decode('utf-8')  
      p = HTMLTableParser()
      p.feed(xhtml)

      position1= p.tables[1][0].index(pos1)
      position2= p.tables[1][0].index(pos2)

      li_1=[p.tables[1][0][position1:],p.tables[1][11][position1:]]
      li_2=[p.tables[1][0][position2:],p.tables[1][11][position2:]]
      
      a=pd.DataFrame(li_1).transpose()
      a.columns=['Year','EPS']
      a['Symbol']=row['Symbol']
      a.replace(['TTM'],'Mar 2021',inplace=True)
      a['Growth_Rate']=a['EPS'].astype(float).pct_change()
      a['Current_EPS']=float(a['EPS'].iloc[-1])
      EPS1=EPS1.append(a,True)

      b=pd.DataFrame(li_2).transpose()
      b.columns=['Year','EPS']
      b['Symbol']=row['Symbol']
      b.replace(['TTM'],'Mar 2021',inplace=True)
      b['Growth_Rate']=b['EPS'].astype(float).pct_change()
      b['Current_EPS']=float(b['EPS'].iloc[-1])
      EPS2=EPS2.append(b,True)
      
    except:
      continue
 
  return EPS1,EPS2   


EPS_7yrs,EPS_4yrs=EPS('Mar 2015','Mar 2018')

EPS_7yrs['Helper']=EPS_7yrs['Symbol']+ " "+ EPS_7yrs['Year']
EPS_4yrs['Helper']=EPS_4yrs['Symbol']+ " "+ EPS_4yrs['Year']   

## **MONTHLY DATA**

In [ ]:
def Monthly_Stock_Price(st):
  Stock_Price=pd.DataFrame()
  end = datetime.datetime.now()
  
  for index, row in nifty_500.iterrows():
    try:
      data = (yf.download(row['Symbol'], start=st, end=end,interval='1mo')['Adj Close']).to_frame()
      data['Symbol']=row['Symbol']
      data = data[data['Adj Close'].notna()]
      Stock_Price=Stock_Price.append(data)
    except:
      continue
  
  return Stock_Price


Stock_Price_7yrs=Monthly_Stock_Price(datetime.datetime(2014,4,1))

Stock_Price_4yrs=Monthly_Stock_Price(datetime.datetime(2017,4,1))

# **Data Preparing**

In [ ]:
def data_Prep(Stock_Price):
  Stock_Price['Year']=Stock_Price.index.year
  Stock_Price['Month']=Stock_Price.index.month
  Stock_Price['Helper']='Mar'
  Stock_Price['Year'] = np.where((Stock_Price.Month >3), Stock_Price.Year + 1, Stock_Price.Year)
  Stock_Price['Helper']= Stock_Price['Symbol']+" "+Stock_Price['Helper'] + " " + Stock_Price["Year"].astype(str)
  Stock_Price.drop(columns=['Year','Month'],inplace=True)
  Stock_Price.reset_index(inplace=True)
  Stock_Price.head()

  return Stock_Price


Stock_Price_4yrs=data_Prep(Stock_Price_4yrs)
Stock_Price_7yrs=data_Prep(Stock_Price_7yrs)

In [ ]:
def Data_Merge(Stock_Price,EPS):
  Final_Data=Stock_Price.merge(EPS,on='Helper',how='left')
  Final_Data=Final_Data[Final_Data['EPS'].notna()]
  Final_Data.drop(columns=['Helper','Symbol_y'],inplace=True)
  Final_Data['P/E Ratio']=Final_Data['Adj Close'].astype(float)/Final_Data['EPS'].astype(float)
  
  return Final_Data

Final_Data_7yrs=Data_Merge(Stock_Price_7yrs,EPS_7yrs)
Final_Data_4yrs=Data_Merge(Stock_Price_4yrs,EPS_4yrs)

In [ ]:
Grouped_Data_7yrs=Final_Data_7yrs.groupby('Symbol_x').mean()
Grouped_Data_7yrs.drop(columns=['Adj Close'],inplace=True)
Grouped_Data_7yrs.rename(columns={'P/E Ratio':'PE_7yrs'},inplace=True)


Grouped_Data_4yrs=Final_Data_4yrs.groupby('Symbol_x').mean()
Grouped_Data_4yrs.drop(columns=['Adj Close'],inplace=True)
Grouped_Data_4yrs.rename(columns={'P/E Ratio':'PE_4yrs'},inplace=True)

In [ ]:
def best_case_scenario(df1,df2):
  df=df1.merge(df2,left_index=True,right_index=True)
  df['Projected EPS']=(1+ df['Growth_Rate'])*df['Current_EPS']
  df['lb_target']=df['Projected EPS']*df['PE_7yrs']
  df['ub target']=df['Projected EPS']*df['PE_4yrs']
  df['fv_best']=(df['lb_target']+df['ub target'])/2
  df['entry_best']=df['fv_best']*0.67
  df=df[['fv_best','entry_best']]
  
  return df
  

best=best_case_scenario(Grouped_Data_7yrs,Grouped_Data_4yrs[['PE_4yrs']])

In [ ]:
def neutral_case_scenario(df1,df2):
  df=df1.merge(df2,left_index=True,right_index=True)
  df['Projected EPS']=(1+ df['Growth_Rate']/2) * df['Current_EPS']
  df['lb_target']=df['Projected EPS']*df['PE_7yrs']
  df['ub target']=df['Projected EPS']*df['PE_4yrs']
  df['fv_neutral']=(df['lb_target']+df['ub target'])/2
  df['entry_neutral']=df['fv_neutral']*0.67
  df=df[['fv_neutral','entry_neutral']]
  
  return df
  

neutral=neutral_case_scenario(Grouped_Data_7yrs,Grouped_Data_4yrs[['PE_4yrs']])

In [ ]:
def poor_case_scenario(df1,df2):
  df=df1.merge(df2,left_index=True,right_index=True)
  df['Projected EPS']=(1- df['Growth_Rate']) * df['Current_EPS']
  df['lb_target']=df['Projected EPS']*df['PE_7yrs']
  df['ub target']=df['Projected EPS']*df['PE_4yrs']
  df['fv_poor']=(df['lb_target']+df['ub target'])/2
  df['entry_poor']=df['fv_poor']*0.67
  df=df[['fv_poor','entry_poor']]
  
  return df
  

poor=poor_case_scenario(Grouped_Data_7yrs,Grouped_Data_4yrs[['PE_4yrs']])

In [ ]:
def last_step(best,neutral,poor):
  best=best.merge(neutral,left_index=True,right_index=True)
  best=best.merge(poor,left_index=True,right_index=True)

  best['QOQ+_FV']=(best['fv_best']+best['fv_neutral'])/2
  best['QOQ+_Entry']=(best['entry_best']+best['entry_neutral'])/2

  best['QOQ-_FV']=(best['fv_best']+best['fv_neutral']+best['fv_poor'])/3
  best['QOQ-_Entry']=(best['entry_best']+best['entry_neutral']+best['entry_poor'])/3

  best=best[['QOQ+_FV','QOQ+_Entry','QOQ-_FV','QOQ-_Entry']]
  
  return best

df_final=last_step(best,neutral,poor)
df_final